## Machine Translation Project
#### Machine Translation is an automated software used to translate a Text from one Natural Language to another.

In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense
import numpy as np

batch_size=64
epochs=100
latent_dim=256
num_samples=10000
data_path = 'C:/Users/91993/French to English Dataset.txt'

### Vectorizing the Data

In [12]:
# Vectorizing the data.

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    
# For Targets, 'tab' is used as start sequence & '\n' for end sequence.    

    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
            
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [16]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))

num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)

max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])

In [17]:
print('Total Number of samples:', len(input_texts))
print('unique input tokens:', num_encoder_tokens)
print('unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Total Number of samples: 10000
unique input tokens: 71
unique output tokens: 93
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [19]:
# Assigning token based on the sorted list

input_token_index=dict(
    [(char,i) for i, char in enumerate(input_characters)])
target_token_index=dict(
[(char,i) for i, char in enumerate(target_characters)])

In [24]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

### One-Hot Encoding

In [25]:
# Now One-Hot Encoding

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    
    for t, char in enumerate(target_text):
# since decoder_target_data is ahead of decoder_input_data by one timestep,
        decoder_input_data[i, t, target_token_index[char]] = 1.
        
        if t > 0:
# Then decoder_target_data will be ahead by one timestep and start ch will not be included
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [29]:
print(encoder_input_data[0].shape)
print(decoder_input_data[0].shape)

(15, 71)
(59, 93)


### Encoder and Decoder

In [32]:

# DEFINING ENCODER AND DECODER 

## Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [34]:
# For Setting up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# Decoder is set to return full output sequence & internal states.

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

### Creating and Training the Model

In [35]:

# Now, taking all these values and creating the model

## Defining model which turn encoder_input_data & decoder_input_data into decoder_target_data 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compiling and Fitting the data
## running training set
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 52s 414ms/step - loss: 1.1564 - accuracy: 0.7332 - val_loss: 1.1157 - val_accuracy: 0.7135
Epoch 2/100
125/125 [==============================] - 49s 391ms/step - loss: 0.8413 - accuracy: 0.7730 - val_loss: 0.8414 - val_accuracy: 0.7601
Epoch 3/100
125/125 [==============================] - 52s 412ms/step - loss: 0.6743 - accuracy: 0.8117 - val_loss: 0.7025 - val_accuracy: 0.7959
Epoch 4/100
125/125 [==============================] - 38s 305ms/step - loss: 0.5794 - accuracy: 0.8326 - val_loss: 0.6320 - val_accuracy: 0.8185
Epoch 5/100
125/125 [==============================] - 39s 310ms/step - loss: 0.5284 - accuracy: 0.8455 - val_loss: 0.6016 - val_accuracy: 0.8232
Epoch 6/100
125/125 [==============================] - 38s 301ms/step - loss: 0.4935 - accuracy: 0.8551 - val_loss: 0.5727 - val_accuracy: 0.8303
Epoch 7/100
125/125 [==============================] - 38s 307ms/step - loss: 0.4659 - accuracy: 0.8624 - val_loss: 0.5535 -

Epoch 57/100
125/125 [==============================] - 32s 259ms/step - loss: 0.0946 - accuracy: 0.9700 - val_loss: 0.5823 - val_accuracy: 0.8736
Epoch 58/100
125/125 [==============================] - 34s 269ms/step - loss: 0.0923 - accuracy: 0.9708 - val_loss: 0.5879 - val_accuracy: 0.8742
Epoch 59/100
125/125 [==============================] - 33s 262ms/step - loss: 0.0905 - accuracy: 0.9713 - val_loss: 0.5949 - val_accuracy: 0.8732
Epoch 60/100
125/125 [==============================] - 33s 266ms/step - loss: 0.0881 - accuracy: 0.9718 - val_loss: 0.6018 - val_accuracy: 0.8734
Epoch 61/100
125/125 [==============================] - 34s 269ms/step - loss: 0.0867 - accuracy: 0.9720 - val_loss: 0.6037 - val_accuracy: 0.8739
Epoch 62/100
125/125 [==============================] - 32s 257ms/step - loss: 0.0849 - accuracy: 0.9728 - val_loss: 0.6072 - val_accuracy: 0.8731
Epoch 63/100
125/125 [==============================] - 32s 254ms/step - loss: 0.0829 - accuracy: 0.9733 - val_loss: 0

In [38]:
model.save('english2frenchmodel.h5')

### Now, Generating the Sentences

In [39]:
# Using LSTM

# Defining sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reversing the lookup token index for decoding sequences back to something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [40]:
def decode_sequence(input_seq):
# Encoding the input as state vectors.
    states_value = encoder_model.predict(input_seq)

# Generating empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    
# Populating the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

# Sampling loop for a batch of sequences (assuming batch size = 1)
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

# Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        
# Exit condition: either hit max length or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

# Updating the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

# Updating states
        states_value = [h, c]

    return decoded_sentence

In [41]:
for seq_index in range(100):
    
# Take one sequence (part of the training set) for decoding

    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Bouge !

-
Input sentence: Go.
Decoded sentence: Bouge !

-
Input sentence: Go.
Decoded sentence: Bouge !

-
Input sentence: Hi.
Decoded sentence: Salut.

-
Input sentence: Hi.
Decoded sentence: Salut.

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded sentence: Cours !

-
Input

In [42]:
# And Done !